In [1]:
import re
import requests
from bs4 import BeautifulSoup
from bs4 import Tag
import pandas as pd
import numpy as np

In [2]:
#URL For Top Free 50 apps
storeUrl = 'https://play.google.com/store/apps/collection/cluster?clp=0g4jCiEKG3RvcHNlbGxpbmdfZnJlZV9BUFBMSUNBVElPThAHGAM%3D:S:ANO1ljKs-KA&gsr=CibSDiMKIQobdG9wc2VsbGluZ19mcmVlX0FQUExJQ0FUSU9OEAcYAw%3D%3D:S:ANO1ljL40zU&hl=en'

html_page = requests.get(storeUrl)
soup = BeautifulSoup(html_page.text,'html.parser')
apps = soup.find_all('div',attrs={'class':'WsMG1c nnK0zc'})
appNames = []
for name in apps:
    appNames.append(name.contents[0])
len(appNames)

50

In [3]:
#Dictionary of all app details
allAppsDetail = {}

#Function to find innermost text in recursive tag
def find_innermost_text(input):
    end = input
    while(isinstance(end, Tag)):
        try:
            end = end.contents[0]
        except:
            return np.NaN
    return end

#Iterate over all the apps
for app in appNames:
    
    #Dictionary to store app details
    details = {}
    
    # Search the app on playstore and crawl the result
    appWithPlus = app.replace(' ','+')
    html_page = requests.get("https://play.google.com/store/search?q="+appWithPlus+"&c=apps")
    soup = BeautifulSoup(html_page.text,'html.parser')
    
    # Extract the href(package name for the app) 
    applink = soup.find('div',attrs={'title':app})
    if(applink==None):
        continue
    applink = applink.parent.attrs['href']
    
    # Open the app specific page
    url='https://play.google.com'+applink
    data=requests.get(url)
    soup = BeautifulSoup(data.text,'html.parser')
    
    # Extract contents from app specific page
    atr_labels = soup.find_all('div',attrs={'class':'BgcNfc'})
    atr_values = soup.find_all(lambda tag: tag.name == 'div' and tag.get('class') == ['IQ1z0d'])
    
    # Fill out app specific details in a dictionary
    for index in range(len(atr_labels)):
        details.update({atr_labels[index].contents[0] : find_innermost_text(atr_values[index])}) 
    
    #Fill Review details
    review_val = int(
            soup.select_one('span[aria-label$="ratings"]').text.replace(",", ""))
    details.update({'Review':review_val})
    
    #Fill Rating details
    rating_val = find_innermost_text(soup.find('div',attrs={'class':'BHMmbe'}))
    details.update({'Rating':rating_val})
    
    #Fill Genre details
    genre_val = find_innermost_text(soup.find('a',attrs={'itemprop':'genre'}))
    details.update({'Genre':genre_val})
            
    allAppsDetail.update({app:details})

In [4]:
result_df = pd.DataFrame(allAppsDetail)
#Transpose result df
apps_df = result_df.T.reset_index().rename(columns={'index':'App'})
apps_df.head(5)

,App,Updated,Size,Installs,Current Version,Requires Android,Content Rating,Interactive Elements,Permissions,Report,Offered By,Developer,Review,Rating,Genre,In-app Products
0,ZOOM Cloud Meetings,"April 8, 2020",16M,"100,000,000+",4.6.20000.0407,5.0 and up,Everyone,Users Interact,View details,Flag as inappropriate,zoom.us,Visit website,92552,4.0,Business,NaN
1,Houseparty,"April 11, 2020",Varies with device,"10,000,000+",1.34.5,4.4 and up,Teen,"Users Interact, Shares Info",View details,Flag as inappropriate,Life on Air,Visit website,101898,2.9,Social,$0.99 - $4.99 per item
2,TikTok - Make Your Day,"April 12, 2020",84M,"1,000,000,000+",15.5.44,4.1 and up,Teen,Users Interact,View details,Flag as inappropriate,TikTok Inc.,Visit website,19007083,4.5,Social,$0.99 - $99.99 per item
3,WhatsApp Messenger,"April 1, 2020",27M,"5,000,000,000+",2.20.108,4.0.3 and up,Everyone,"Users Interact, Digital Purchases",View details,Flag as inappropriate,WhatsApp Inc.,Visit website,109265458,4.3,Communication,NaN
4,Messenger – Text and Video Chat for Free,"April 8, 2020",Varies with device,"1,000,000,000+",Varies with device,Varies with device,Everyone,"Users Interact, Shares Location, Digital Purch...",View details,Flag as inappropriate,Facebook,Visit website,71798565,4.2,Communication,$0.99 - $399.99 per item


In [5]:
# Write to csv
#apps_df.to_csv('dataset/apps_additional_info.csv',index=False)